In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import torch

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from dataset import Dataset
from model import NeuralNetwork
import torch.nn.functional as F


In [3]:
class Dataset(Dataset):

    def __init__(self):
        #self.csv = pd.read_csv('./mnist_train.csv')
        self.csv = pd.read_csv('./a-z_train.csv')

    def __len__(self):
        return len(self.csv) - 160

    def __getitem__(self, ind):
        X = self.csv.iloc[ind + 160][1:].values
        Y = self.csv.iloc[ind + 160][0]

        X = torch.from_numpy(X).float()

        return X, Y



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        
        self.conv2_drop = nn.Dropout2d()
        
        # Size is 784
        
        self.fc1 = nn.Linear(20 * 28 * 28 , 50)
        self.fc2 = nn.Linear(50, 26)

    def forward(self, input):
        print(input.size())
        print(input)
        x = F.relu(F.max_pool2d(self.conv1(input), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        print(x.size())
        
        x = x.view(-1, 20 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x)


In [5]:
images = pd.read_csv('./a-z_train.csv')
#print(images)
#print(images.shape)

In [8]:
batch_size = 64
train_dataset = Dataset()

data_loader = DataLoader(train_dataset,
                         shuffle=True,
                         batch_size=batch_size)
#print (data_loader.shape)
#for i in range(len(sample_data_loader):
#    data_loader = 


print("Loading dataset!")
net = NeuralNetwork()
print("DONE!")
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),
                      lr=0.01,
                      momentum=0.5)

nepoch = 3
for epoch in range(nepoch):
    running_loss = 0.0
    for X, Y in data_loader:
        optimizer.zero_grad()
        #X.values.reshape(28, 28)
        pred = net(X)

        loss = criterion(pred, Y)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    running_loss /= len(data_loader)
    print(running_loss)

torch.save(net.state_dict(), './model.pth')


Loading dataset!
DONE!
torch.Size([64, 784])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 10 1, but got 2-dimensional input of size [64, 784] instead